# Remarks

* Data normalization
    * Mobilenet expects data from -1 to 1
        * Normalize Input Data or Include in Model
        * TFLite Conversion must fit according to decision
    * Ground Truth Data: for better inspection Data multiplied by 80. Undo the change in the Data Input Pipeline
* Overview in Tutorials:
    * tf.function
* Idea

# Libraries

In [1]:
#!for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done

import tensorflow as tf
import numpy as np
import os
import time
from datetime import datetime
import sys
import matplotlib.pyplot as plt
import argparse
import yaml

import DataProcessing.dataset as dataset
import DataProcessing.model_files as model_files
import Nets.backbones as backbones
import Nets.features as features
import Nets.losses as losses
import Nets.metrics as metrics
import Nets.visualize as visualize
import Nets.tools as tools

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

#np.set_printoptions(threshold=sys.maxsize)

# Parser

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--model', type=str, required=False, default=None)
parser.add_argument('--data', type=str, required=False, default=None)

parser.add_argument('--bs', type=int, required=False, default=None)
parser.add_argument('--idx', type=int, required=False, default=None)
parser.add_argument('--epoch', type=int, required=False, default=None)
parser.add_argument('--noise', type=float, required=False, default=None)

parser.add_argument('--train_model', action='store_true', default=False)
parser.add_argument('--cache', action='store_true', default=False)
parser.add_argument('--save', action='store_true', default=False)
parser.add_argument('--sigmoid', action='store_true', default=False)
parser.add_argument('--focal', action='store_true', default=False)

parser.add_argument('--beta_upper', type=float, required=False, default=None)
parser.add_argument('--gamma', type=float, required=False, default=None)
parser.add_argument('--alpha', type=float, required=False, default=None)

file_name = None
try:
    file_name = __file__
except:
    print("Jupyter Notebook")
       
if file_name is None:
    args = parser.parse_args("")
    args.train_model = True
    args.cache = True
    #args.save = True
    args.save = True
    args.sigmoid = False
    args.focal = True
else:    
    args = parser.parse_args()

Jupyter Notebook


# Options

In [3]:
# Generall Parameters
TRAIN_MODEL = True #args.train_model
SEED = None

# LOSS
weighted_multi_label_sigmoid_edge_loss = args.sigmoid
# focal_loss = args.focal
focal_loss = True

beta_upper = 0.5 if args.beta_upper is None else args.beta_upper
beta_lower = 1.0 - beta_upper
gamma = 2.0 if args.gamma is None else args.gamma
alpha = 2.0 if args.alpha is None else args.alpha
class_weighted = True
weighted_beta = True

config_path = os.path.join(os.getcwd(), 'configs')
model_config_path = os.path.join(config_path, 'model.yaml')

with open(model_config_path, 'r') as file:
    model_cfg = yaml.safe_load(file)

# Load Dataset, Preprocess Images and Dataset

In [4]:
tf.random.set_seed(SEED)

DP = dataset.DataProcessing(model_cfg["INPUT_SHAPE"], model_cfg["OUTPUT_SHAPE"], config_path)
DP.path_definitions()

MF = model_files.ModelFiles()
MF.path_definitions(model_cfg["NAME"], DP.ds_cfg["NAME"], make_dirs=True)
MF.clean_model_directories(model_cfg["CALLBACKS"]["DEL_OLD_CKPT"], model_cfg["CALLBACKS"]["DEL_OLD_TB"])

if TRAIN_MODEL:
    rng = tf.random.Generator.from_seed(123, alg='philox')

    train_ds, img_count_train = DP.load_dataset(DP.key.train)
    train_ds = DP.dataset_processing(train_ds, DP.key.train, shuffle=True, prefetch=True, img_count=img_count_train,
                                     rng=rng)

test_ds, img_count_test = DP.load_dataset(DP.key.test)
test_ds = DP.dataset_processing(test_ds, DP.key.test, shuffle=False, prefetch=True, img_count=img_count_test)


2022-11-02 10:14:47.705995: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-02 10:14:47.706041: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (david-ThinkPad-X1-Yoga-Gen-6): /proc/driver/nvidia/version does not exist
2022-11-02 10:14:47.706584: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TRAIN 1
The TRAIN DS contains 202 images.
TEST 2
The TEST DS contains 2 images.


# Model

In [5]:
if weighted_multi_label_sigmoid_edge_loss:
    loss = lambda y_true, y_pred: losses.weighted_multi_label_sigmoid_loss(y_true, y_pred, beta_lower=beta_lower,
                                                                           beta_upper=beta_upper,
                                                                           class_weighted=class_weighted)
elif focal_loss:
    loss = lambda y_true, y_pred: losses.focal_loss_edges(y_true, y_pred, gamma=gamma, alpha=alpha,
                                                          weighted_beta=weighted_beta, beta_lower=beta_lower,
                                                          beta_upper=beta_upper, class_weighted=class_weighted)
else:
    raise ValueError("either FocalLoss or WeightedMultiLabelSigmoidLoss must be True")

In [7]:
if TRAIN_MODEL:
    output_dims = model_cfg["OUTPUT_SHAPE"]

    # BACKBONE
    backbone, output_names = backbones.get_backbone(name=model_cfg["BACKBONE"]["NAME"],
                                                    weights=model_cfg["BACKBONE"]["WEIGHTS"],
                                                    height=model_cfg["INPUT_SHAPE"][0],
                                                    width=model_cfg["INPUT_SHAPE"][1],
                                                    alpha=model_cfg["BACKBONE"]["ALPHA"],
                                                    output_layer=model_cfg["BACKBONE"]["OUTPUT_IDS"],
                                                    trainable_idx=model_cfg["BACKBONE"]["TRAIN_IDX"])
    
    # prior edge map input
    ann_input = tf.keras.Input(shape=(output_dims[0], output_dims[1],1), name='input_PRIOR_ANN')
    x = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same",
                      strides=1, use_bias=False)(ann_input)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    
    x = tf.keras.layers.DepthwiseConv2D(kernel_size=5, dilation_rate=1, padding="same",
                                        strides=4, use_bias=False)(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x_out = tf.keras.layers.ReLU(name="x_out")(x)
    
    # backbone output
    b = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                               strides=1, use_bias=False)(backbone.output[-1])
    b = tf.keras.layers.BatchNormalization()(b)
    b = tf.keras.layers.ReLU()(b)
    
    # Mix layers
    #concat:
    c = tf.keras.layers.Concatenate(axis=-1)([x_out,b])
    c = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same",
                                    strides=1, use_bias=False)(c)
    c = tf.keras.layers.BatchNormalization()(c)
    c = tf.keras.layers.ReLU()(c)
    c = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same",
                                strides=1, use_bias=False)(c)
    c = tf.keras.layers.BatchNormalization()(c)
    c = tf.keras.layers.ReLU()(c)
    
    # mult. layers:
    mu = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same", 
                           strides=1, use_bias=False)(backbone.output[-1])
    mu = tf.keras.layers.BatchNormalization()(mu)
    mu = tf.keras.layers.ReLU()(mu)
    
    mu = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                       strides=1, use_bias=False)(mu)
    mu = tf.keras.layers.BatchNormalization()(mu)
    mu = tf.keras.layers.ReLU(name="out_mu")(mu)
    
    out = []
    for i in range(9):
        out.append(mu*c[:,:,:,i:i+1])
    x = tf.keras.layers.Concatenate(axis=-1)(out)
    
    # Post Processing
    x = tf.keras.layers.DepthwiseConv2D(kernel_size=3, dilation_rate=1, padding="same",
                                        strides=1, use_bias=False)(x)
    
    x = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same",
                               strides=1, use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    
    comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                                 strides=1, use_bias=True)(x)
    comp = tf.keras.layers.BatchNormalization()(comp)
    
    edge_comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                                 strides=1, use_bias=True)(ann_input)
    
    edge_comp = tf.keras.layers.BatchNormalization()(edge_comp)
    edge_comp = tf.keras.layers.ReLU()(edge_comp)
    
    edge_comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                                       strides=2, use_bias=True)(edge_comp)
    edge_comp = tf.keras.layers.BatchNormalization()(edge_comp)
    edge_comp = tf.keras.layers.ReLU()(edge_comp)
    
    edge_comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                                   strides=1, use_bias=True)(edge_comp)
    edge_comp = tf.keras.layers.BatchNormalization()(edge_comp)
    edge_comp = tf.keras.layers.ReLU()(edge_comp)
    
    edge_comp = tf.keras.layers.MaxPool2D(pool_size=(3, 3),strides=2,padding='same')(edge_comp)
    
    #INPUT EDGE POINTS:
    num_points = 20
    vert_input = tf.keras.Input(shape=(num_points,3),name='input_PRIOR_VERT')
    
    x = tf.keras.layers.Conv1D(9,1,strides=1, padding="same", activation="relu")(vert_input)
    x = tf.keras.layers.Conv1D(9,1,strides=1, padding="same", activation="relu")(x)
    x_split = tf.split(x, 20, axis=1)
    
    
    comp = tf.keras.layers.Concatenate(axis=-1)([comp, edge_comp])
    comp = tf.keras.layers.Conv2D(9, kernel_size=3, dilation_rate=1, padding="same", 
                               strides=1, use_bias=True)(comp)
    comp = tf.keras.layers.BatchNormalization()(comp)
    comp = tf.keras.layers.ReLU()(comp)
    
    comp = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same", 
                           strides=1, use_bias=True)(comp)
    comp = tf.keras.layers.BatchNormalization()(comp)
    comp_sep = tf.keras.layers.ReLU()(comp)
    
    comp = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same", 
                       strides=1, use_bias=True)(comp_sep)
    comp = tf.keras.layers.BatchNormalization()(comp)
    comp = tf.keras.layers.ReLU()(comp)
    
    comp = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same", 
                   strides=1, use_bias=True)(comp)
    comp = tf.keras.layers.BatchNormalization()(comp)
    comp = tf.keras.layers.ReLU()(comp)
    
    comp = tf.keras.layers.Concatenate(axis=-1)([comp, comp_sep])
    
    comp = tf.keras.layers.Conv2D(16, kernel_size=3, dilation_rate=1, padding="same")(comp)
    
    output_comp = comp
    
    # taking a batchsize of one:
    row_len = 2
    col_len = 2
    shape_comp = tf.shape(comp)
    shape_comp = tf.cast(shape_comp, tf.float32)
    comp = tf.pad(comp, tf.constant([[0,0],[row_len, row_len],[col_len, col_len],[0,0]]),"CONSTANT")
    shape = tf.shape(vert_input)

    out_row = []
    for row in tf.range(shape[1]):
        #co_row = tf.math.round(vert_input[0, row, 1]*shape_comp[1])
        #co_col = tf.math.round(vert_input[0, row, 0]*shape_comp[2])
        #co_row = tf.cast(co_row, tf.int32)
        #co_col = tf.cast(co_col, tf.int32)
        co_row = tf.cast(vert_input[0, row, 1]*shape_comp[1], tf.int32)
        co_col = tf.cast(vert_input[0, row, 0]*shape_comp[2], tf.int32)
        c = tf.slice(comp,begin=[0,co_row, co_col, 0],size=[-1,2*row_len+1,2*col_len+1, -1])
        out_row.append(c)
    
    out = []
    for i in range(len(out_row)):
        l1 = tf.keras.layers.Flatten()(out_row[i])
        l2 = tf.keras.layers.Flatten()(x_split[i])
        l = tf.keras.layers.Concatenate(axis=-1)([l1,l2])
        l = tf.keras.layers.Dense(6, activation="relu")(l)
        l = tf.keras.layers.Dense(6, activation="relu")(l)
        l = tf.keras.layers.Concatenate(axis=-1)([l1,l2,l])
        l = tf.keras.layers.Dense(6, activation="relu")(l)
        l = tf.keras.layers.Dense(3, activation="relu")(l)
        l = tf.reshape(l,[-1,1,3])
        out.append(l)
    output = tf.keras.layers.Concatenate(axis=1, name = "output_CURRENT_VERT")(out)   
    
    # CONCATENATE
    #side_outputs = [upsample_side_1, upsample_side_2, decoded]
    # concat = features.shared_concatenation(side_outputs,NUM_CLASSES)
    # output = features.fused_classification(concat,NUM_CLASSES,name="output")
    #output = features.shared_concatenation_fused_classification(side_outputs, DP.num_classes, name="ANN")
    model = tf.keras.Model(inputs=(backbone.input, ann_input, vert_input), outputs=[output])

    model.layers[-1]._name = "output_CURRENT_VERT"
    model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_IMG (InputLayer)         [(None, 640, 360, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_PRIOR_ANN (InputLayer)   [(None, 320, 180, 1  0           []                               
                                )]                                                                
                                                                                                  
 tf.math.truediv_1 (TFOpLambda)  (None, 640, 360, 3)  0          ['input_IMG[0][0]']              
                                                                                              

 tf.__operators__.getitem_112 (  (None, 80, 45, 1)   0           ['re_lu_16[0][0]']               
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__operators__.getitem_113 (  (None, 80, 45, 1)   0           ['re_lu_16[0][0]']               
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__operators__.getitem_114 (  (None, 80, 45, 1)   0           ['re_lu_16[0][0]']               
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__operators__.getitem_115 (  (None, 80, 45, 1)   0           ['re_lu_16[0][0]']               
 SlicingOp

                                                                                                  
 batch_normalization_23 (BatchN  (None, 80, 45, 16)  64          ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_26 (Conv2D)             (None, 160, 90, 9)   738         ['re_lu_20[0][0]']               
                                                                                                  
 re_lu_18 (ReLU)                (None, 80, 45, 16)   0           ['batch_normalization_23[0][0]'] 
                                                                                                  
 batch_normalization_27 (BatchN  (None, 160, 90, 9)  36          ['conv2d_26[0][0]']              
 ormalization)                                                                                    
          

                                                                                                  
 tf.__operators__.getitem_140 (  ()                  0           ['tf.range_1[0][0]']             
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__operators__.getitem_145 (  ()                  0           ['tf.range_1[0][0]']             
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__operators__.getitem_150 (  ()                  0           ['tf.range_1[0][0]']             
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__oper

                                                                                                  
 tf.__operators__.getitem_133 (  ()                  0           ['input_PRIOR_VERT[0][0]',       
 SlicingOpLambda)                                                 'tf.__operators__.getitem_130[0]
                                                                 [0]']                            
                                                                                                  
 tf.__operators__.getitem_134 (  ()                  0           ['tf.cast_41[0][0]']             
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__operators__.getitem_136 (  ()                  0           ['input_PRIOR_VERT[0][0]',       
 SlicingOpLambda)                                                 'tf.__operators__.getitem_135[0]
          

...

 flatten_55 (Flatten)           (None, 9)            0           ['tf.split_1[0][7]']             
                                                                                                  
 flatten_56 (Flatten)           (None, 400)          0           ['tf.slice_28[0][0]']            
                                                                                                  
 flatten_57 (Flatten)           (None, 9)            0           ['tf.split_1[0][8]']             
                                                                                                  
 flatten_58 (Flatten)           (None, 400)          0           ['tf.slice_29[0][0]']            
                                                                                                  
 flatten_59 (Flatten)           (None, 9)            0           ['tf.split_1[0][9]']             
                                                                                                  
 flatten_6

 concatenate_70 (Concatenate)   (None, 409)          0           ['flatten_62[0][0]',             
                                                                  'flatten_63[0][0]']             
                                                                                                  
 concatenate_72 (Concatenate)   (None, 409)          0           ['flatten_64[0][0]',             
                                                                  'flatten_65[0][0]']             
                                                                                                  
 concatenate_74 (Concatenate)   (None, 409)          0           ['flatten_66[0][0]',             
                                                                  'flatten_67[0][0]']             
                                                                                                  
 concatenate_76 (Concatenate)   (None, 409)          0           ['flatten_68[0][0]',             
          

 dense_113 (Dense)              (None, 6)            42          ['dense_112[0][0]']              
                                                                                                  
 dense_117 (Dense)              (None, 6)            42          ['dense_116[0][0]']              
                                                                                                  
 dense_121 (Dense)              (None, 6)            42          ['dense_120[0][0]']              
                                                                                                  
 dense_125 (Dense)              (None, 6)            42          ['dense_124[0][0]']              
                                                                                                  
 dense_129 (Dense)              (None, 6)            42          ['dense_128[0][0]']              
                                                                                                  
 dense_133

                                                                                                  
 concatenate_79 (Concatenate)   (None, 415)          0           ['flatten_70[0][0]',             
                                                                  'flatten_71[0][0]',             
                                                                  'dense_141[0][0]']              
                                                                                                  
 concatenate_81 (Concatenate)   (None, 415)          0           ['flatten_72[0][0]',             
                                                                  'flatten_73[0][0]',             
                                                                  'dense_145[0][0]']              
                                                                                                  
 concatenate_83 (Concatenate)   (None, 415)          0           ['flatten_74[0][0]',             
          

 dense_127 (Dense)              (None, 3)            21          ['dense_126[0][0]']              
                                                                                                  
 dense_131 (Dense)              (None, 3)            21          ['dense_130[0][0]']              
                                                                                                  
 dense_135 (Dense)              (None, 3)            21          ['dense_134[0][0]']              
                                                                                                  
 dense_139 (Dense)              (None, 3)            21          ['dense_138[0][0]']              
                                                                                                  
 dense_143 (Dense)              (None, 3)            21          ['dense_142[0][0]']              
                                                                                                  
 dense_147

# Compile and Train Model

In [8]:
for inp, out in train_ds.take(1):
    print(inp.keys())
    print(out.keys())

dict_keys(['input_PRIOR_ANN', 'input_IMG', 'input_PRIOR_VERT'])
dict_keys(['output_CURRENT_VERT'])


In [10]:
if TRAIN_MODEL:
    # learning rate schedule
    base_learning_rate = 0.15*100
    end_learning_rate = 0.05*100
    decay_step = np.ceil(img_count_train / DP.ds_cfg[DP.key.train]["BATCH_SIZE"]) * model_cfg["EPOCHS"]
    lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(base_learning_rate, decay_steps=decay_step,
                                                                end_learning_rate=end_learning_rate, power=0.9)
    
    frequency = int(
        np.ceil(img_count_train / DP.ds_cfg[DP.key.train]["BATCH_SIZE"]) * model_cfg["CALLBACKS"]["CKPT_FREQ"]) + 1

    logdir = os.path.join(MF.paths['TBLOGS'], datetime.now().strftime("%Y%m%d-%H%M%S"))
    callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath=MF.paths["CKPT"] + "/ckpt-loss={val_loss:.2f}-epoch={epoch:.2f}",
        save_weights_only=False, save_best_only=False, monitor="val_f1", verbose=1, save_freq='epoch', period=frequency),
        tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)]

    # compile model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  loss={'output_CURRENT_VERT': losses.coordinate_loss},)
                  #metrics={'output_CURRENT_VERT': [metrics.BinaryAccuracyEdges(threshold_prediction=0),
                  #         metrics.F1Edges(threshold_prediction=0, threshold_edge_width=0)]})

    history = model.fit(train_ds, epochs=model_cfg["EPOCHS"], validation_data=test_ds, callbacks=callbacks)


Epoch 1/120
201/201 [==============================] - 29s 123ms/step - loss: 0.2414 - val_loss: 0.2606
Epoch 2/120
201/201 [==============================] - 27s 135ms/step - loss: 0.2414 - val_loss: 0.2606
Epoch 3/120
201/201 [==============================] - 28s 139ms/step - loss: 0.2414 - val_loss: 0.2606
Epoch 4/120
201/201 [==============================] - 27s 131ms/step - loss: 0.2414 - val_loss: 0.2606
Epoch 5/120
201/201 [==============================] - 36s 177ms/step - loss: 0.2414 - val_loss: 0.2606
Epoch 6/120
201/201 [==============================] - 28s 139ms/step - loss: 0.2414 - val_loss: 0.2606
Epoch 7/120
201/201 [==============================] - 29s 142ms/step - loss: 0.2414 - val_loss: 0.2606
Epoch 8/120
201/201 [==============================] - 26s 127ms/step - loss: 0.2414 - val_loss: 0.2606
Epoch 9/120
201/201 [==============================] - 26s 127ms/step - loss: 0.2414 - val_loss: 0.2606
Epoch 10/120
201/201 [==============================] - 25s 126m

KeyboardInterrupt: 

In [ ]:
model_ckpt = os.listdir(MF.paths['CKPT'])

f1_max = 0
for ckpt_name in model_ckpt:
    if float(ckpt_name[-4:]) > f1_max:
        f1_max = float(ckpt_name[-4:])
        model_path = MF.paths['CKPT'] + "/" + ckpt_name

        print(model_path)

custom_objects = {"BinaryAccuracyEdges": metrics.BinaryAccuracyEdges, "F1Edges": metrics.F1Edges, "<lambda>": loss}

model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

# Plot Results

In [ ]:
if TRAIN_MODEL:
    plot_losses = ["loss", "loss"]
    plot_metrics = ["accuracy_edges", "f1", "recall", "precision"]

    path = os.path.join(MF.paths["FIGURES"], "training.svg")

    visualize.plot_training_results(res=history.history, losses=plot_losses, metrics=plot_metrics,
                                    save=model_cfg["SAVE"], path=path)

In [ ]:
### Maximum F1 Score:
# TODO: FAILS right now: fix
# if not TRAIN_MODEL:
#     step_width = 0.05
#     threshold_range = [0.05, 0.95]
#     threshold_array = np.arange(threshold_range[0], threshold_range[1] + step_width, step_width)
#     threshold_array = np.array([0.025, 0.1, 0.2, 0.3, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.975])
#
#     path_metrics_evaluation_plot = os.path.join(MF.paths["FIGURES"], "threshold_metrics_evaluation_test_ds.svg")
#     threshold_f1_max = visualize.plot_threshold_metrics_evaluation_class(model=model, ds=test_ds,
#                                                                          num_classes=DP.num_classes,
#                                                                          threshold_array=threshold_array,
#                                                                          threshold_edge_width=0, save=model_cfg["SAVE"],
#                                                                          path=path_metrics_evaluation_plot)

# Fine Tuning

# Evaluate on Test DS of Real Images

## Metrics Evaluation

## Visual Results

# Save Model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=loss,
                  metrics={'output': [metrics.BinaryAccuracyEdges(threshold_prediction=0),
                                      metrics.F1Edges(threshold_prediction=0, threshold_edge_width=0)]})

if model_cfg["SAVE"]:
    model.save(MF.paths["MODEL"])
    
    custom_objects = {"BinaryAccuracyEdges": metrics.BinaryAccuracyEdges,
                      "F1Edges": metrics.F1Edges,
                      "<lambda>":loss}
    
    model = tf.keras.models.load_model(MF.paths["MODEL"], custom_objects=custom_objects)